In [19]:
import torch
from torchvision import models, transforms
from PIL import Image
import requests
import time

# Load labels
LABELS_URL = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
labels = requests.get(LABELS_URL).text.strip().split("\n")

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transform pipeline
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [20]:
def classify_image(img_path):
    image = Image.open(img_path).convert("RGB")
    input_tensor = transform(image).unsqueeze(0).to(device)

    model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT).to(device)
    model.eval()

    with torch.no_grad():
        start = time.time()
        output = model(input_tensor)
        end = time.time()

    probs = torch.nn.functional.softmax(output[0], dim=0)
    top5 = torch.topk(probs, 5)

    print(f"\nInference device: {device}")
    print(f"Time taken: {end - start:.4f} seconds\n")
    for i in range(5):
        print(f"{labels[top5.indices[i]]}: {top5.values[i].item()*100:.2f}%")

In [22]:
from io import BytesIO

# Grab an image from the web and classify it
# Use a direct JPG link so PIL can open it
img_url = "https://upload.wikimedia.org/wikipedia/commons/9/9a/Pug_600.jpg"

# Download the image
try:
    response = requests.get(img_url)
    image = Image.open(BytesIO(response.content)).convert("RGB")

# Basic error handling to make sure we're getting an image
except: 
    print(response.status_code)
    print(response.headers['Content-Type'])
    print(response.text[:200])  # Preview first 200 characters


# Run it through the classifier
classify_image(image)


Inference device: cpu
Time taken: 0.0504 seconds

pug: 38.80%
silky terrier: 0.87%
bull mastiff: 0.76%
chow: 0.58%
Norwegian elkhound: 0.44%
